## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Luderitz,NaN,-26.6481,15.1594,57.18,75,66,14.72,broken clouds
1,1,Constitucion,CL,-35.3333,-72.4167,56.17,63,0,11.32,clear sky
2,2,Hithadhoo,MV,-0.6000,73.0833,82.22,73,45,7.27,scattered clouds
3,3,Airai,TL,-8.9266,125.4092,56.75,58,93,2.10,overcast clouds
4,4,Hobart,AU,-42.8794,147.3294,41.27,85,2,5.75,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your vacation? "))
max_temp = float(input("What is the maximum temperature you would like for your vacation? "))

What is the minimum temperature you would like for your vacation? 70
What is the maximum temperature you would like for your vacation? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)] 
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Hithadhoo,MV,-0.6000,73.0833,82.22,73,45,7.27,scattered clouds
5,5,San Quintin,MX,30.4833,-115.9500,79.72,62,35,13.31,scattered clouds
7,7,Nabire,ID,-3.3667,135.4833,74.39,90,29,3.18,scattered clouds
10,10,Kahului,US,20.8947,-156.4700,82.98,66,40,21.85,scattered clouds
12,12,Vaini,TO,-21.2000,-175.2000,77.16,94,20,10.36,few clouds
14,14,Sorong,ID,-0.8833,131.2500,77.58,85,51,4.36,broken clouds
15,15,Dingle,PH,10.9995,122.6711,75.52,89,97,1.43,overcast clouds
16,16,Les Cayes,HT,18.2000,-73.7500,85.86,68,60,7.31,broken clouds
17,17,Korla,CN,41.7597,86.1469,88.90,24,34,13.56,scattered clouds
18,18,Hambantota,LK,6.1241,81.1185,77.34,94,20,5.75,few clouds


In [5]:
#4a. Determine if there are any empty rows. 
preferred_cities_df.count()

City_ID                347
City                   347
Country                345
Lat                    347
Lng                    347
Max Temp               347
Humidity               347
Cloudiness             347
Wind Speed             347
Current Description    347
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
vacation_cities_df=preferred_cities_df.dropna()

In [7]:
vacation_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Hithadhoo,MV,-0.6000,73.0833,82.22,73,45,7.27,scattered clouds
5,5,San Quintin,MX,30.4833,-115.9500,79.72,62,35,13.31,scattered clouds
7,7,Nabire,ID,-3.3667,135.4833,74.39,90,29,3.18,scattered clouds
10,10,Kahului,US,20.8947,-156.4700,82.98,66,40,21.85,scattered clouds
12,12,Vaini,TO,-21.2000,-175.2000,77.16,94,20,10.36,few clouds


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = vacation_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Hithadhoo,MV,82.22,scattered clouds,-0.6000,73.0833,
5,San Quintin,MX,79.72,scattered clouds,30.4833,-115.9500,
7,Nabire,ID,74.39,scattered clouds,-3.3667,135.4833,
10,Kahului,US,82.98,scattered clouds,20.8947,-156.4700,
12,Vaini,TO,77.16,few clouds,-21.2000,-175.2000,
14,Sorong,ID,77.58,broken clouds,-0.8833,131.2500,
15,Dingle,PH,75.52,overcast clouds,10.9995,122.6711,
16,Les Cayes,HT,85.86,broken clouds,18.2000,-73.7500,
17,Korla,CN,88.90,scattered clouds,41.7597,86.1469,
18,Hambantota,LK,77.34,few clouds,6.1241,81.1185,


In [18]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
 
    # 6c. Get latitude and longitude from DataFrame
    params["locations"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    

In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= '']

In [19]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation(1).csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name


In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [14]:
# 11a. Add a marker layer for each city to the map. 
marker = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))